# Tuning and Optimizing Neural Networks - Lab

## Introduction

Now that you've practiced regularization, initialization, and optimization techniques, its time to synthesize these concepts into a cohesive modeling pipeline.  

With this pipeline, you will not only fit an initial model but also attempt to improve it. Your final model selection will pertain to the test metrics across these models. This will more naturally simulate a problem you might be faced with in practice, and the various modeling decisions you are apt to encounter along the way.  

Recall that our end objective is to achieve a balance between overfitting and underfitting. You've seen the bias variance trade-off, and the role of regularization in order to reduce overfitting on training data and improving generalization to new cases. Common frameworks for such a procedure include train/validate/test methodology when data is plentiful, and K-folds cross-validation for smaller, more limited datasets. In this lab, you'll perform the latter, as the dataset in question is fairly limited. 

## Objectives

You will be able to:

* Apply normalization as a preprocessing technique 
* Implement a K-folds cross validation modeling pipeline for deep learning models 
* Apply regularization techniques to improve your model's performance 

## Load the data

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [27]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_predict
from keras import models
from keras import layers
from keras import regularizers
from keras.wrappers.scikit_learn import KerasRegressor

In this lab you'll be working with the *The Lending Club* data. 

- Import the data available in the file `'loan_final.csv'` 
- Drop rows with missing values in the `'total_pymnt'` column (this is your target column) 
- Print the first five rows of the data 
- Print the dimensions of the data 

In [28]:
# Import the data
data = pd.read_csv('loan_final.csv')

# Drop rows with no target value
data = data[data['total_pymnt'].notna()]

# Print the first five rows
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


In [29]:
# Print the dimensions of data 
data.head()

,loan_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_length,home_ownership,annual_inc,verification_status,loan_status,purpose,addr_state,total_acc,total_pymnt,application_type
0,5000.0,4975.0,36 months,10.65%,162.87,B,10+ years,RENT,24000.0,Verified,Fully Paid,credit_card,AZ,9.0,5863.155187,Individual
1,2500.0,2500.0,60 months,15.27%,59.83,C,< 1 year,RENT,30000.0,Source Verified,Charged Off,car,GA,4.0,1014.530000,Individual
2,2400.0,2400.0,36 months,15.96%,84.33,C,10+ years,RENT,12252.0,Not Verified,Fully Paid,small_business,IL,10.0,3005.666844,Individual
3,10000.0,10000.0,36 months,13.49%,339.31,C,10+ years,RENT,49200.0,Source Verified,Fully Paid,other,CA,37.0,12231.890000,Individual
4,3000.0,3000.0,60 months,12.69%,67.79,B,1 year,RENT,80000.0,Source Verified,Fully Paid,other,OR,38.0,4066.908161,Individual


## Generating a Hold Out Test Set

While we will be using K-fold cross validation to select an optimal model, we still want a final hold out test set that is completely independent of any modeling decisions. As such, pull out a sample of 30% of the total available data. For consistency of results, use random seed 42. 

In [30]:
from sklearn.model_selection import train_test_split

# Features to build the model
features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc', 
            'home_ownership', 'verification_status', 'emp_length']

X = data[features]
y = data[['total_pymnt']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.30, random_state=42)

## Preprocessing (Numerical features) 

- Fill all missing values in numeric features with their respective means 
- Standardize all the numeric features  
- Convert the final results into DataFrames 

In [34]:
# Select continuous features
cont_features = ['loan_amnt', 'funded_amnt_inv', 'installment', 'annual_inc']

X_train_cont = X_train.loc[:, cont_features]
X_test_cont = X_test.loc[:, cont_features]

# Instantiate SimpleImputer - fill the missing values with the mean
si = SimpleImputer(missing_values=np.nan, strategy='mean')

# Fit and transform the training data
X_train_imputed = si.fit_transform(X_train_cont)

# Transform test data
X_test_imputed = si.transform(X_test_cont)

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform the training data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_imputed), columns=cont_features)

# Transform test data
X_test_scaled = pd.DataFrame(ss_X.fit_transform(X_test_imputed), columns=cont_features)

## Preprocessing (Categorical features) 

- Fill all missing values in categorical features with the string `'missing'` 
- One-hot encode all categorical features 
- Convert the final results into DataFrames 


In [35]:
# Select only the categorical features
cat_features = ['home_ownership', 'verification_status', 'emp_length']
X_train_cat = X_train.loc[:, cat_features]
X_test_cat = X_test.loc[:, cat_features]

# Fill missing values with the string 'missing'
X_train_cat.fillna('missing', inplace=True)
X_test_cat.fillna('missing', inplace=True)

# OneHotEncode categorical variables
ohe = OneHotEncoder(handle_unknown='ignore')

# Transform training and test sets
X_train_ohe = ohe.fit_transform(X_train_cat)
X_test_ohe = ohe.fit_transform(X_test_cat)

# Get all categorical feature names
cat_columns = ohe.get_feature_names(input_features=X_train_cat.columns)

# Fit and transform the training data
X_train_categorical = pd.DataFrame(X_train_ohe.todense(), columns=cat_columns)

# Transform test data
X_test_categorical = pd.DataFrame(X_test_ohe.todense(), columns=cat_columns)

Run the below cell to combine the numeric and categorical features. 

In [36]:
# Combine continuous and categorical feature DataFrames
X_train_all = pd.concat([X_train_scaled, X_train_categorical], axis=1)
X_test_all = pd.concat([X_test_scaled, X_test_categorical], axis=1)

# Number of input features
n_features = X_train_all.shape[1]

- Standardize the target DataFrames (`y_train` and `y_test`) 

In [37]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform Y (train)
y_train_scaled = ss_y.fit_transform(y_train)

# Transform test Y (test)
y_test_scaled = ss_y.fit_transform(y_test)

## Define a K-fold Cross Validation Methodology

Now that your have a complete holdout test set, you will perform k-fold cross-validation using the following steps: 

- Create a function that returns a compiled deep learning model 
- Use the wrapper function `KerasRegressor()` that defines how these folds are trained 
- Call the `cross_val_predict()` function to perform k-fold cross-validation 

In the cell below, we've defined a baseline model that returns a compiled Keras models. 

In [38]:
# Define a function that returns a compiled Keras model 
def create_baseline_model():
    
    # Initialize model
    model = models.Sequential()

    # First hidden layer
    model.add(layers.Dense(10, activation='relu', input_shape=(n_features,)))

    # Second hidden layer
    model.add(layers.Dense(5, activation='relu'))

    # Output layer
    model.add(layers.Dense(1, activation='linear'))

    # Compile the model
    model.compile(optimizer='SGD', 
                  loss='mse',  
                  metrics=['mse']) 
    
    # Return the compiled model
    return model

Wrap `create_baseline_model` inside a call to `KerasRegressor()`, and: 

- Train for 150 epochs 
- Set the batch size to 256 

> NOTE: Refer to the [documentation](https://keras.io/scikit-learn-api/) to learn about `KerasRegressor()`.  

In [39]:
# Wrap the above function for use in cross-validation
keras_wrapper_1 = KerasRegressor(create_baseline_model, epochs=150, batch_size=256)

Use `cross_val_predict()` to generate cross-validated predictions with: 
- 5-fold cv 
- scaled input (`X_train_all`) and output (`y_train_scaled`) 

In [40]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_baseline_preds = cross_val_predict(keras_wrapper_1, X_train_all, y_train_scaled, cv=5)

Epoch 1/150
23819/23819 [==============================] - 0s 9us/step - loss: 0.5846 - mean_squared_error: 0.5846
Epoch 2/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2697 - mean_squared_error: 0.2697
Epoch 3/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2476 - mean_squared_error: 0.2476
Epoch 4/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2360 - mean_squared_error: 0.2360
Epoch 5/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2286 - mean_squared_error: 0.2286
Epoch 6/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2238 - mean_squared_error: 0.2238
Epoch 7/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2200 - mean_squared_error: 0.2200
Epoch 8/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2173 - mean_squared_error: 0.2173
Epoch 9/150
23819/23819 [==============================] - 0s 5us/step - loss: 0

23819/23819 [==============================] - 0s 5us/step - loss: 0.1985 - mean_squared_error: 0.1985
Epoch 72/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1982 - mean_squared_error: 0.1982
Epoch 73/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1982 - mean_squared_error: 0.1982
Epoch 74/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1982 - mean_squared_error: 0.1982
Epoch 75/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1980 - mean_squared_error: 0.1980
Epoch 76/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1978 - mean_squared_error: 0.1978
Epoch 77/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1978 - mean_squared_error: 0.1978
Epoch 78/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1978 - mean_squared_error: 0.1978
Epoch 79/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.197

23819/23819 [==============================] - 0s 5us/step - loss: 0.1942 - mean_squared_error: 0.1942
Epoch 142/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1941 - mean_squared_error: 0.1941
Epoch 143/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1938 - mean_squared_error: 0.1938
Epoch 144/150
23819/23819 [==============================] - 0s 4us/step - loss: 0.1937 - mean_squared_error: 0.1937
Epoch 145/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1937 - mean_squared_error: 0.1937
Epoch 146/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1937 - mean_squared_error: 0.1937
Epoch 147/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1935 - mean_squared_error: 0.1935
Epoch 148/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1935 - mean_squared_error: 0.1935
Epoch 149/150
23819/23819 [==============================] - 0s 5us/step - los

23819/23819 [==============================] - 0s 4us/step - loss: 0.2037 - mean_squared_error: 0.2037
Epoch 62/150
23819/23819 [==============================] - 0s 4us/step - loss: 0.2037 - mean_squared_error: 0.2037
Epoch 63/150
23819/23819 [==============================] - 0s 4us/step - loss: 0.2033 - mean_squared_error: 0.2033
Epoch 64/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2033 - mean_squared_error: 0.2033
Epoch 65/150
23819/23819 [==============================] - 0s 4us/step - loss: 0.2032 - mean_squared_error: 0.2032
Epoch 66/150
23819/23819 [==============================] - 0s 4us/step - loss: 0.2031 - mean_squared_error: 0.2031
Epoch 67/150
23819/23819 [==============================] - 0s 4us/step - loss: 0.2031 - mean_squared_error: 0.2031
Epoch 68/150
23819/23819 [==============================] - 0s 4us/step - loss: 0.2029 - mean_squared_error: 0.2029
Epoch 69/150
23819/23819 [==============================] - 0s 4us/step - loss: 0.203

23819/23819 [==============================] - 0s 5us/step - loss: 0.1981 - mean_squared_error: 0.1981
Epoch 132/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1981 - mean_squared_error: 0.1981
Epoch 133/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1980 - mean_squared_error: 0.1980
Epoch 134/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1979 - mean_squared_error: 0.1979
Epoch 135/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1978 - mean_squared_error: 0.1978
Epoch 136/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1979 - mean_squared_error: 0.1979
Epoch 137/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1978 - mean_squared_error: 0.1978
Epoch 138/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1978 - mean_squared_error: 0.1978
Epoch 139/150
23819/23819 [==============================] - 0s 5us/step - los

...

23819/23819 [==============================] - 0s 5us/step - loss: 0.1941 - mean_squared_error: 0.1941
Epoch 122/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1939 - mean_squared_error: 0.1939
Epoch 123/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1940 - mean_squared_error: 0.1940
Epoch 124/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1941 - mean_squared_error: 0.1941
Epoch 125/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1939 - mean_squared_error: 0.1939
Epoch 126/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1937 - mean_squared_error: 0.1937
Epoch 127/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1939 - mean_squared_error: 0.1939
Epoch 128/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1937 - mean_squared_error: 0.1937
Epoch 129/150
23819/23819 [==============================] - 0s 5us/step - los

23819/23819 [==============================] - 0s 6us/step - loss: 0.2049 - mean_squared_error: 0.2049
Epoch 42/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2047 - mean_squared_error: 0.2047
Epoch 43/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2044 - mean_squared_error: 0.2044
Epoch 44/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2042 - mean_squared_error: 0.2042
Epoch 45/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2042 - mean_squared_error: 0.2042
Epoch 46/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2042 - mean_squared_error: 0.2042
Epoch 47/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2038 - mean_squared_error: 0.2038
Epoch 48/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2037 - mean_squared_error: 0.2037
Epoch 49/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.203

23819/23819 [==============================] - 0s 5us/step - loss: 0.1976 - mean_squared_error: 0.1976
Epoch 112/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1978 - mean_squared_error: 0.1978
Epoch 113/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1975 - mean_squared_error: 0.1975
Epoch 114/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1974 - mean_squared_error: 0.1974
Epoch 115/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1973 - mean_squared_error: 0.1973
Epoch 116/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1972 - mean_squared_error: 0.1972
Epoch 117/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1970 - mean_squared_error: 0.1970
Epoch 118/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1972 - mean_squared_error: 0.1972
Epoch 119/150
23819/23819 [==============================] - 0s 5us/step - los

23820/23820 [==============================] - 0s 5us/step - loss: 0.2057 - mean_squared_error: 0.2057
Epoch 32/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.2055 - mean_squared_error: 0.2055
Epoch 33/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.2052 - mean_squared_error: 0.2052
Epoch 34/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.2051 - mean_squared_error: 0.2051
Epoch 35/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.2046 - mean_squared_error: 0.2046
Epoch 36/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.2045 - mean_squared_error: 0.2045
Epoch 37/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.2041 - mean_squared_error: 0.2041
Epoch 38/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.2040 - mean_squared_error: 0.2040
Epoch 39/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.203

23820/23820 [==============================] - 0s 5us/step - loss: 0.1967 - mean_squared_error: 0.1967
Epoch 102/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1966 - mean_squared_error: 0.1966
Epoch 103/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1965 - mean_squared_error: 0.1965
Epoch 104/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1965 - mean_squared_error: 0.1965
Epoch 105/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1964 - mean_squared_error: 0.1964
Epoch 106/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1964 - mean_squared_error: 0.1964
Epoch 107/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1963 - mean_squared_error: 0.1963
Epoch 108/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1962 - mean_squared_error: 0.1962
Epoch 109/150
23820/23820 [==============================] - 0s 5us/step - los

- Find the RMSE on train data 

In [41]:
# RMSE on train data (scaled)
MSE = mean_squared_error(y_train_scaled, cv_baseline_preds)
RMSE = np.sqrt(MSE)
print(RMSE)

0.4441978511702158


- Convert the scaled predictions back to original scale 
- Calculate RMSE in the original units with `y_train` and `baseline_preds` 

In [42]:
# Convert the predictions back to original scale
baseline_preds = ss_y.inverse_transform(cv_baseline_preds)

# RMSE on train data (original scale)
MSE = mean_squared_error(y_train, baseline_preds)
RMSE = np.sqrt(MSE)
print(RMSE)

4046.9457705189457


## Intentionally Overfitting a Model

Now that you've developed a baseline model, its time to intentionally overfit a model. To overfit a model, you can:
* Add layers
* Make the layers bigger
* Increase the number of training epochs

Again, be careful here. Think about the limitations of your resources, both in terms of your computers specs and how much time and patience you have to let the process run. Also keep in mind that you will then be regularizing these overfit models, meaning another round of experiments and more time and resources.

In [46]:
# Define a function that returns a compiled Keras model 
def create_bigger_model():
    m = models.Sequential()
    
    m.add(layers.Dense(10, activation='tanh', input_shape=(n_features,)))
    m.add(layers.Dense(10, activation='tanh'))
    m.add(layers.Dense(10, activation='tanh'))
    m.add(layers.Dense(1, activation='linear'))

    m.compile(optimizer='SGD', loss='mse', metrics=['mse']) 
    
    return m


In [47]:
# Wrap the above function for use in cross-validation
keras_wrapper_2 = KerasRegressor(create_bigger_model, epochs=150, batch_size=256)

In [48]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_bigger_model_preds = cross_val_predict(keras_wrapper_2, X_train_all, y_train_scaled, cv=5)

Epoch 1/150
23819/23819 [==============================] - 0s 13us/step - loss: 0.4620 - mean_squared_error: 0.4620
Epoch 2/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.3172 - mean_squared_error: 0.3172
Epoch 3/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2883 - mean_squared_error: 0.2883
Epoch 4/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2710 - mean_squared_error: 0.2710
Epoch 5/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2590 - mean_squared_error: 0.2590
Epoch 6/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.2502 - mean_squared_error: 0.2502
Epoch 7/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.2435 - mean_squared_error: 0.2435
Epoch 8/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.2376 - mean_squared_error: 0.2376
Epoch 9/150
23819/23819 [==============================] - 0s 5us/step - loss: 

23819/23819 [==============================] - 0s 5us/step - loss: 0.1931 - mean_squared_error: 0.1931
Epoch 72/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1929 - mean_squared_error: 0.1929
Epoch 73/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1926 - mean_squared_error: 0.1926
Epoch 74/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1927 - mean_squared_error: 0.1927
Epoch 75/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1924 - mean_squared_error: 0.1924
Epoch 76/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1925 - mean_squared_error: 0.1925
Epoch 77/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1924 - mean_squared_error: 0.1924
Epoch 78/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1924 - mean_squared_error: 0.1924
Epoch 79/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.192

23819/23819 [==============================] - 0s 5us/step - loss: 0.1888 - mean_squared_error: 0.1888
Epoch 142/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1885 - mean_squared_error: 0.1885
Epoch 143/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1886 - mean_squared_error: 0.1886
Epoch 144/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1886 - mean_squared_error: 0.1886
Epoch 145/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1884 - mean_squared_error: 0.1884
Epoch 146/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1885 - mean_squared_error: 0.1885
Epoch 147/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1886 - mean_squared_error: 0.1886
Epoch 148/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1886 - mean_squared_error: 0.1886
Epoch 149/150
23819/23819 [==============================] - 0s 5us/step - los

23819/23819 [==============================] - 0s 5us/step - loss: 0.1999 - mean_squared_error: 0.1999
Epoch 62/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1999 - mean_squared_error: 0.1999
Epoch 63/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1994 - mean_squared_error: 0.1994
Epoch 64/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1993 - mean_squared_error: 0.1993
Epoch 65/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1992 - mean_squared_error: 0.1992
Epoch 66/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.1990 - mean_squared_error: 0.1990
Epoch 67/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.1990 - mean_squared_error: 0.1990
Epoch 68/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1987 - mean_squared_error: 0.1987
Epoch 69/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.198

23819/23819 [==============================] - 0s 5us/step - loss: 0.1952 - mean_squared_error: 0.1952
Epoch 132/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1952 - mean_squared_error: 0.1952
Epoch 133/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1952 - mean_squared_error: 0.1952
Epoch 134/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1952 - mean_squared_error: 0.1952
Epoch 135/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1950 - mean_squared_error: 0.1950
Epoch 136/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1951 - mean_squared_error: 0.1951
Epoch 137/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1952 - mean_squared_error: 0.1952
Epoch 138/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1952 - mean_squared_error: 0.1952
Epoch 139/150
23819/23819 [==============================] - 0s 5us/step - los

...

23819/23819 [==============================] - 0s 5us/step - loss: 0.1897 - mean_squared_error: 0.1897
Epoch 122/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1895 - mean_squared_error: 0.1895
Epoch 123/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1899 - mean_squared_error: 0.1899
Epoch 124/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1895 - mean_squared_error: 0.1895
Epoch 125/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1893 - mean_squared_error: 0.1893
Epoch 126/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1892 - mean_squared_error: 0.1892
Epoch 127/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1895 - mean_squared_error: 0.1895
Epoch 128/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1892 - mean_squared_error: 0.1892
Epoch 129/150
23819/23819 [==============================] - 0s 5us/step - los

23819/23819 [==============================] - 0s 5us/step - loss: 0.1993 - mean_squared_error: 0.1993
Epoch 42/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1990 - mean_squared_error: 0.1990
Epoch 43/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1996 - mean_squared_error: 0.1996
Epoch 44/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1985 - mean_squared_error: 0.1985
Epoch 45/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1982 - mean_squared_error: 0.1982
Epoch 46/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1983 - mean_squared_error: 0.1983
Epoch 47/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1976 - mean_squared_error: 0.1976
Epoch 48/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1972 - mean_squared_error: 0.1972
Epoch 49/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.197

23819/23819 [==============================] - 0s 5us/step - loss: 0.1907 - mean_squared_error: 0.1907
Epoch 112/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1909 - mean_squared_error: 0.1909
Epoch 113/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1908 - mean_squared_error: 0.1908
Epoch 114/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1906 - mean_squared_error: 0.1906
Epoch 115/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1906 - mean_squared_error: 0.1906
Epoch 116/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1908 - mean_squared_error: 0.1908
Epoch 117/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1905 - mean_squared_error: 0.1905
Epoch 118/150
23819/23819 [==============================] - 0s 5us/step - loss: 0.1906 - mean_squared_error: 0.1906
Epoch 119/150
23819/23819 [==============================] - 0s 5us/step - los

23820/23820 [==============================] - 0s 5us/step - loss: 0.2010 - mean_squared_error: 0.2010
Epoch 32/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.2007 - mean_squared_error: 0.2007
Epoch 33/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.2003 - mean_squared_error: 0.2003
Epoch 34/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1997 - mean_squared_error: 0.1997
Epoch 35/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1993 - mean_squared_error: 0.1993
Epoch 36/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1990 - mean_squared_error: 0.1990
Epoch 37/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1984 - mean_squared_error: 0.1984
Epoch 38/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1981 - mean_squared_error: 0.1981
Epoch 39/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.197

23820/23820 [==============================] - 0s 5us/step - loss: 0.1901 - mean_squared_error: 0.1901
Epoch 102/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1896 - mean_squared_error: 0.1896
Epoch 103/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1897 - mean_squared_error: 0.1897
Epoch 104/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1896 - mean_squared_error: 0.1896
Epoch 105/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1896 - mean_squared_error: 0.1896
Epoch 106/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1896 - mean_squared_error: 0.1896
Epoch 107/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1895 - mean_squared_error: 0.1895
Epoch 108/150
23820/23820 [==============================] - 0s 5us/step - loss: 0.1895 - mean_squared_error: 0.1895
Epoch 109/150
23820/23820 [==============================] - 0s 5us/step - los

In [49]:
# RMSE on train data (scaled)
MSE = mean_squared_error(y_train, cv_bigger_model_preds)
RMSE = np.sqrt(MSE)
print(RMSE)

15077.863258917949


## Regularizing the Model to Achieve Balance  

Now that you have a powerful model (albeit an overfit one), we can now increase the generalization of the model by using some of the regularization techniques we discussed. Some options you have to try include:  
* Adding dropout
* Adding L1/L2 regularization
* Altering the layer architecture (add or remove layers similar to above)  

This process will be constrained by time and resources. Be sure to test at least two different methodologies, such as dropout and L2 regularization. If you have the time, feel free to continue experimenting. 

In [53]:
# Define a function that returns a compiled Keras model 
def create_regularized_model():
    
    m = models.Sequential()
    m.add(layers.Dropout(0.3, input_shape=(n_features,)))
    m.add(layers.Dense(10, activation='tanh', input_shape=(n_features,)))
    m.add(layers.Dropout(0.3))
    m.add(layers.Dense(10, kernel_regularizer=regularizers.l2(0.005), activation='tanh'))
    m.add(layers.Dense(10, activation='tanh'))
    m.add(layers.Dense(1, kernel_regularizer=regularizers.l2(0.005), activation='linear'))

    m.compile(optimizer='SGD', loss='mse', metrics=['mse']) 
    
    return m

In [54]:
# Wrap the above function for use in cross-validation
keras_wrapper_3 = KerasRegressor(create_regularized_model, epochs=150, batch_size=256)

In [55]:
# ⏰ This cell may take several mintes to run
# Generate cross-validated predictions
np.random.seed(123)
cv_dropout_preds = cross_val_predict(keras_wrapper_3, X_train_all, y_train_scaled, cv=5)

Epoch 1/150
23819/23819 [==============================] - 0s 18us/step - loss: 0.6662 - mean_squared_error: 0.6011
Epoch 2/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.5204 - mean_squared_error: 0.4575
Epoch 3/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.4857 - mean_squared_error: 0.4245
Epoch 4/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.4664 - mean_squared_error: 0.4067
Epoch 5/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.4554 - mean_squared_error: 0.3971
Epoch 6/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.4493 - mean_squared_error: 0.3922
Epoch 7/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.4321 - mean_squared_error: 0.3764
Epoch 8/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.4243 - mean_squared_error: 0.3698
Epoch 9/150
23819/23819 [==============================] - 0s 6us/step - loss: 

23819/23819 [==============================] - 0s 7us/step - loss: 0.3460 - mean_squared_error: 0.3269
Epoch 72/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.3479 - mean_squared_error: 0.3291
Epoch 73/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.3520 - mean_squared_error: 0.3334
Epoch 74/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3499 - mean_squared_error: 0.3316
Epoch 75/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3436 - mean_squared_error: 0.3255
Epoch 76/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3395 - mean_squared_error: 0.3215
Epoch 77/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3438 - mean_squared_error: 0.3260
Epoch 78/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3395 - mean_squared_error: 0.3219
Epoch 79/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.345

23819/23819 [==============================] - 0s 6us/step - loss: 0.3269 - mean_squared_error: 0.3165
Epoch 142/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3302 - mean_squared_error: 0.3198
Epoch 143/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3307 - mean_squared_error: 0.3204
Epoch 144/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3269 - mean_squared_error: 0.3167
Epoch 145/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3265 - mean_squared_error: 0.3164
Epoch 146/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.3232 - mean_squared_error: 0.3131
Epoch 147/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3264 - mean_squared_error: 0.3164
Epoch 148/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3283 - mean_squared_error: 0.3183
Epoch 149/150
23819/23819 [==============================] - 0s 7us/step - los

23819/23819 [==============================] - 0s 6us/step - loss: 0.3551 - mean_squared_error: 0.3367
Epoch 62/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3563 - mean_squared_error: 0.3382
Epoch 63/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3542 - mean_squared_error: 0.3363
Epoch 64/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3512 - mean_squared_error: 0.3336
Epoch 65/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3559 - mean_squared_error: 0.3385
Epoch 66/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3535 - mean_squared_error: 0.3363
Epoch 67/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.3541 - mean_squared_error: 0.3372
Epoch 68/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.3565 - mean_squared_error: 0.3399
Epoch 69/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.354

23819/23819 [==============================] - 0s 6us/step - loss: 0.3353 - mean_squared_error: 0.3257
Epoch 132/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3374 - mean_squared_error: 0.3278
Epoch 133/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3365 - mean_squared_error: 0.3269
Epoch 134/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3338 - mean_squared_error: 0.3243
Epoch 135/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3384 - mean_squared_error: 0.3290
Epoch 136/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3261 - mean_squared_error: 0.3167
Epoch 137/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3305 - mean_squared_error: 0.3211
Epoch 138/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3368 - mean_squared_error: 0.3274
Epoch 139/150
23819/23819 [==============================] - 0s 6us/step - los

...

23819/23819 [==============================] - 0s 6us/step - loss: 0.3322 - mean_squared_error: 0.3212
Epoch 122/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.3287 - mean_squared_error: 0.3177
Epoch 123/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3309 - mean_squared_error: 0.3200
Epoch 124/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3301 - mean_squared_error: 0.3194
Epoch 125/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3287 - mean_squared_error: 0.3179
Epoch 126/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3337 - mean_squared_error: 0.3231
Epoch 127/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3312 - mean_squared_error: 0.3206
Epoch 128/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3321 - mean_squared_error: 0.3216
Epoch 129/150
23819/23819 [==============================] - 0s 6us/step - los

23819/23819 [==============================] - 0s 6us/step - loss: 0.3709 - mean_squared_error: 0.3431
Epoch 42/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3669 - mean_squared_error: 0.3395
Epoch 43/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3716 - mean_squared_error: 0.3448
Epoch 44/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3646 - mean_squared_error: 0.3381
Epoch 45/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3654 - mean_squared_error: 0.3393
Epoch 46/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3688 - mean_squared_error: 0.3432
Epoch 47/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3625 - mean_squared_error: 0.3372
Epoch 48/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.3666 - mean_squared_error: 0.3417
Epoch 49/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.364

23819/23819 [==============================] - 0s 7us/step - loss: 0.3386 - mean_squared_error: 0.3258
Epoch 112/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.3326 - mean_squared_error: 0.3200
Epoch 113/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3409 - mean_squared_error: 0.3284
Epoch 114/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3345 - mean_squared_error: 0.3220
Epoch 115/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3342 - mean_squared_error: 0.3218
Epoch 116/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3341 - mean_squared_error: 0.3218
Epoch 117/150
23819/23819 [==============================] - 0s 7us/step - loss: 0.3390 - mean_squared_error: 0.3268
Epoch 118/150
23819/23819 [==============================] - 0s 6us/step - loss: 0.3329 - mean_squared_error: 0.3207
Epoch 119/150
23819/23819 [==============================] - 0s 7us/step - los

23820/23820 [==============================] - 0s 7us/step - loss: 0.3746 - mean_squared_error: 0.3474
Epoch 32/150
23820/23820 [==============================] - 0s 7us/step - loss: 0.3713 - mean_squared_error: 0.3447
Epoch 33/150
23820/23820 [==============================] - 0s 7us/step - loss: 0.3732 - mean_squared_error: 0.3470
Epoch 34/150
23820/23820 [==============================] - 0s 7us/step - loss: 0.3725 - mean_squared_error: 0.3468
Epoch 35/150
23820/23820 [==============================] - 0s 6us/step - loss: 0.3721 - mean_squared_error: 0.3468
Epoch 36/150
23820/23820 [==============================] - 0s 6us/step - loss: 0.3691 - mean_squared_error: 0.3442
Epoch 37/150
23820/23820 [==============================] - 0s 6us/step - loss: 0.3684 - mean_squared_error: 0.3439
Epoch 38/150
23820/23820 [==============================] - 0s 7us/step - loss: 0.3637 - mean_squared_error: 0.3396
Epoch 39/150
23820/23820 [==============================] - 0s 6us/step - loss: 0.364

23820/23820 [==============================] - 0s 6us/step - loss: 0.3342 - mean_squared_error: 0.3222
Epoch 102/150
23820/23820 [==============================] - 0s 6us/step - loss: 0.3311 - mean_squared_error: 0.3191
Epoch 103/150
23820/23820 [==============================] - 0s 6us/step - loss: 0.3376 - mean_squared_error: 0.3256
Epoch 104/150
23820/23820 [==============================] - 0s 6us/step - loss: 0.3323 - mean_squared_error: 0.3205
Epoch 105/150
23820/23820 [==============================] - 0s 6us/step - loss: 0.3267 - mean_squared_error: 0.3149
Epoch 106/150
23820/23820 [==============================] - 0s 6us/step - loss: 0.3358 - mean_squared_error: 0.3242
Epoch 107/150
23820/23820 [==============================] - 0s 6us/step - loss: 0.3310 - mean_squared_error: 0.3194
Epoch 108/150
23820/23820 [==============================] - 0s 6us/step - loss: 0.3269 - mean_squared_error: 0.3153
Epoch 109/150
23820/23820 [==============================] - 0s 6us/step - los

In [56]:
# RMSE on train data (scaled)
MSE = mean_squared_error(y_train, cv_dropout_preds)
RMSE = np.sqrt(MSE)
print(RMSE)

15077.878529026284


## Summary

In this lab, you investigated some data from *The Lending Club* in a complete data science pipeline to build neural networks with good performance. You began with reserving a hold-out set for testing which never was touched during the modeling phase. From there, you implemented a k-fold cross validation methodology in order to assess an initial baseline model and various regularization methods. 